In [34]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [9]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [10]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [11]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [12]:
np.random.seed(42)

sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

class Net:
    def __init__(self, weights_and_biases = ()):
        if len(weights_and_biases) == 0:
            self.w1 = np.random.uniform(-3, 3, (9, 9))
            self.w2 = np.random.uniform(-3, 3, (9, 1))
            self.b1 = np.random.uniform(-3, 3, (9, 1))
            self.b2 = np.random.uniform(-3, 3, 1)
        else:
            self.w1 = weights_and_biases[0]
            self.w2 = weights_and_biases[1]
            self.b1 = weights_and_biases[2]
            self.b2 = weights_and_biases[3]
        self.mae = 0
    
    def set_weights(self, weights_and_biases):
        self.w1 = weights_and_biases[0]
        self.w2 = weights_and_biases[1]
        self.b1 = weights_and_biases[2]
        self.b2 = weights_and_biases[3]

    def predict(self, input):
        return (self.w2.T @ sigmoid(self.w1 @ input.T + self.b1) + self.b2).reshape(-1)

    def __str__(self):
        return f"w1:\n{self.w1}\nw2:\n{self.w2}\nb1:\n{self.b1}\nb2:\n{self.b2}"

    def __add__(self, other):
        w1 = (self.w1 + other.w1) / 2 + np.random.normal(0, 3, (9, 9))
        w2 = (self.w2 + other.w2) / 2 + np.random.normal(0, 3, (9, 1))
        b1 = (self.b1 + other.b1) / 2 + np.random.normal(0, 3, (9, 1))
        b2 = (self.b2 + other.b2) / 2 + np.random.normal(0, 3, 1)

        return w1, w2, b1, b2

In [13]:
nets = {}
y_pred = {}

for i in range(100):
    nets[i] = Net()

In [30]:
for key, val in nets.items():
    y_pred[key] = nets[key].predict(scaled_X_train)

mean_absolute_errors = {}

for key, val in y_pred.items():
    mean_absolute_errors[key] = mean_absolute_error(y_train, y_pred[key])

sorted_indecies = [key for key, value in sorted(mean_absolute_errors.items(), key = lambda x: x[1])]

print(sorted(mean_absolute_errors.items(), key = lambda x: x[1])[0])

(97, 3.4242206497323875)


In [29]:
for i in range(0, 50, 2):
    nets[sorted_indecies[50 + i]].set_weights(nets[sorted_indecies[i]] + nets[sorted_indecies[i + 1]])
    nets[sorted_indecies[50 + i + 1]].set_weights(nets[sorted_indecies[i]] + nets[sorted_indecies[i + 1]])

In [31]:
y_pred_test = nets[97].predict(scaled_X_test)

In [33]:
mean_absolute_error(y_test, y_pred_test)

3.498459122221945

In [44]:
def sigmoidt(x):
    test = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))
    return 1 / (1 + math.exp(-x))

t = np.array([2, 3, 5])

sigmoidt(t)

TypeError: only size-1 arrays can be converted to Python scalars